### **Gesture Recognition**

In [0]:
#################################################### CNN + RNN ############################################################################

In [0]:
## Following code covers 
# 1. Transfer learning using Resnet50
# 2. RNN Model Implementation using LSTM to which the Resnet output is attached 

# Due to practical challenges associated with use of Paperspace, a few custom lines of code for using Google Colab have been provide 
# and commented out.
# Please uncomment the same if there is a need to execute the code in Google Colab

In [0]:
## Google Colab specific Code

In [0]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -q keras

In [0]:
# !pip install PyDrive

    100% |████████████████████████████████| 993kB 7.5MB/s 
  Running setup.py bdist_wheel for PyDrive ... - done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
# import os
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials



In [0]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [0]:
# download = drive.CreateFile({'id': '1-p9WJBGw4IO4NXcFNkg3vuuV4bzY5j37'})
# download.GetContentFile('Neural_Nets.zip')

In [0]:
# !unzip Neural_Nets.zip

Archive:  Neural_Nets.zip
   creating: Project_data/
   creating: Project_data/train/
  inflating: Project_data/train.csv  
   creating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00014.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00016.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00018.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00020.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00022.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00024.png  
  inflating: Proj

In [0]:
#################### Actual coding commences here ####################

In [0]:
# Import the required libraries

In [0]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

In [0]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In [0]:
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines()[:663])
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines()[:100])
batch_size = 27#experiment with the batch size

In [0]:
import skimage.transform
from skimage import io


In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D


In [0]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
#     img_idx = #create a list of image numbers you want to use for a particular video
    # We selected all 30 frames in the video since that provided the best accuracy
    img_idx = list(range(0, 30))
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size
        for batch in range(num_batches): # we iterate over the number of batches
#             batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            # Selected the image size as 197x197
            batch_data = np.zeros((batch_size, len(img_idx), 197, 197, 3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]) #.astype(np.float32)
                    
                    image_resized = skimage.transform.resize(image, (197, 197))#, anti_aliasing=True)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:, :, 0] - image_resized[:, :, 0].mean())/ image_resized[:, :, 0].std()
                    batch_data[folder,idx,:,:,1] = (image_resized[:, :, 1] - image_resized[:, :, 1].mean())/ image_resized[:, :, 1].std()
                    batch_data[folder,idx,:,:,2] = (image_resized[:, :, 2] - image_resized[:, :, 2].mean())/ image_resized[:, :, 2].std()

                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        batch = num_batches 

        folders_covered_size = num_batches * batch_size
        folders_remain_size = len(t) - folders_covered_size

        batch_data = np.zeros((folders_remain_size, len(img_idx), 197, 197, 3)) 
        batch_labels = np.zeros((folders_remain_size,5))
        
        for folder in range(folders_remain_size):
            imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = io.imread(source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item]) #.astype(np.float32)
                    
                image_resized = skimage.transform.resize(image, (197, 197))#, anti_aliasing=True)
                    
                batch_data[folder,idx,:,:,0] = (image_resized[:, :, 0] - image_resized[:, :, 0].mean())/ image_resized[:, :, 0].std()
                batch_data[folder,idx,:,:,1] = (image_resized[:, :, 1] - image_resized[:, :, 1].mean())/ image_resized[:, :, 1].std()
                batch_data[folder,idx,:,:,2] = (image_resized[:, :, 2] - image_resized[:, :, 2].mean())/ image_resized[:, :, 2].std()
                
                    
            batch_labels[folder, int(t[folder].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
          

In [0]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
# num_epochs = # choose the number of epochs
num_epochs = 12
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 12


In [0]:
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model

We will implement the convolutional neural network part by leveraging transfer learning through use of Resnet50 followed by LSTM layer

In [0]:
base_model = ResNet50(weights = 'imagenet', include_top = False,input_shape= (197, 197,3))
print(base_model.input)
print(base_model.summary())

Tensor("input_2:0", shape=(?, 197, 197, 3), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 203, 203, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________

In [0]:
# training the model after 140 layers
split_at = 140
for layer in base_model.layers[:split_at]: layer.trainable = False
for layer in base_model.layers[split_at:]: layer.trainable = True

In [0]:
base_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate when we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

base_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



In [0]:
#transfer learning: getting the output from base model

base_model_output = base_model.output
print(base_model_output)

Tensor("avg_pool_1/AvgPool:0", shape=(?, 1, 1, 2048), dtype=float32)


In [0]:
x = GlobalAveragePooling2D()(base_model_output)
x = Dense(512, activation = 'relu')(x)


In [0]:
cnn = Model(inputs=base_model.input, outputs = x)
print(cnn.output)

Tensor("dense_4/Relu:0", shape=(?, 512), dtype=float32)


In [0]:
cnn.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [0]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# LR = # write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

In [0]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [0]:
cnn.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
from keras.engine import Input
from keras.layers import LSTM

In [0]:
from keras.regularizers import l2
#LSTM
model = Sequential()

# Add the output of the resnet model into the timedistributed layer
model.add(TimeDistributed(cnn, batch_input_shape=(None, 30 ,197,197,3)))

# Add an LSTM layer containing 128 neurons
model.add(LSTM(128, return_sequences=True))

# Flatten the output
model.add(Flatten())

# Add a fully connected dense layer along with regularizer to avoid overfitting
model.add(Dense(256,kernel_regularizer=l2(0.1)))

model.add(Dense(5, activation='softmax'))

print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (None, 30, 512)           24636800  
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           328192    
_________________________________________________________________
flatten_4 (Flatten)          (None, 3840)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               983296    
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 1285      
Total params: 25,949,573
Trainable params: 17,337,861
Non-trainable params: 8,611,712
_________________________________________________________________
None


In [0]:
# Choose SGD as the optimizer with a learning rate of 0.0005
optimiser = optimizers.SGD(lr=0.0005, decay = 1e-6, nesterov = True, momentum=0.9)

model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])





In [0]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [0]:
# Add early stopping and decay LR through callbacks if the validation loss fails to reduce and plateaus
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR, EarlyStopping(monitor='val_loss', patience=5, verbose=0)]

In [0]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [0]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


Source path = Epoch 1/12 
/content/Project_data/train ; batch size = 27


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


25/25 [==============================] - 470s 19s/step - loss: 48.6583 - categorical_accuracy: 0.4840 - val_loss: 47.4341 - val_categorical_accuracy: 0.5600

Epoch 00001: saving model to model_init_2018-10-1921_58_48.581780/model-00001-48.68332-0.47964-47.43408-0.56000.h5
Epoch 2/12
25/25 [==============================] - 412s 16s/step - loss: 46.0438 - categorical_accuracy: 0.7872 - val_loss: 45.1425 - val_categorical_accuracy: 0.5900

Epoch 00002: saving model to model_init_2018-10-1921_58_48.581780/model-00002-46.06530-0.78582-45.14251-0.59000.h5
Epoch 3/12
25/25 [==============================] - 427s 17s/step - loss: 43.6825 - categorical_accuracy: 0.8039 - val_loss: 43.0944 - val_categorical_accuracy: 0.5900

Epoch 00003: saving model to model_init_2018-10-1921_58_48.581780/model-00003-43.69924-0.80392-43.09440-0.59000.h5
Epoch 4/12
25/25 [==============================] - 428s 17s/step - loss: 41.4908 - categorical_accuracy: 0.8402 - val_loss: 40.7603 - val_categorical_accuracy

In [0]:
cp 'model_init_2018-10-1921_58_48.581780/model-00012-27.43289-0.99849-27.58465-0.71000.h5' '/content/drive/My Drive/'

The generaged model can be accessed from the Google drive link below:
https://drive.google.com/open?id=1-21uomQOFyJF3DF_LO5txNeVkmKrIUzO